In [2]:
# Import Dependencies
import requests
import pprint
import pandas as pd
# Import the EIA API key
from config import EIA_api_key

In [3]:
# Define the API key and endpoint URL
api_key = EIA_api_key

# Set the query parameters

# URL endpoint
url = f"https://api.eia.gov/v2/electricity/state-electricity-profiles/capability/data/?api_key={api_key}&frequency=annual&data[0]=capability&facets[stateId][]=TX&start=1990&end=2012&sort[0][column]=period&sort[0][direction]=desc&sort[1][column]=producertypeid&sort[1][direction]=desc&offset=0&length=5000"

try:
    # Send the API request
    response = requests.get(url)
    response.raise_for_status()

    # Retrieve the data
    data = response.json()

    # Process the data as needed
    if "data" in data["response"]:
        # Extract the generator counts by location and energy source
        generator_counts = data["response"]["data"]
        pprint.pprint(data)
        
    else:
        print("No data available.")
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")

{'apiVersion': '2.1.4',
 'request': {'command': '/v2/electricity/state-electricity-profiles/capability/data/',
             'params': {'api_key': 'x6MMEhX0n69TOHYg6HfanNHKFatn9wv68dbefLC7',
                        'data': ['capability'],
                        'end': 2012,
                        'facets': {'stateId': ['TX']},
                        'frequency': 'annual',
                        'length': 5000,
                        'offset': 0,
                        'sort': [{'column': 'period', 'direction': 'desc'},
                                 {'column': 'producertypeid',
                                  'direction': 'desc'}],
                        'start': 1990}},
 'response': {'data': [{'capability': 109568.4,
                        'capability-units': 'megawatts',
                        'energySourceDescription': 'All',
                        'energysourceid': 'ALL',
                        'period': 2012,
                        'producerTypeDescription': 'All se

In [4]:
# Convert JSON file to Pandas Dataframe and confirm # of rows matches original file
data_df = pd.json_normalize(data["response"]["data"])
print("Rows in Dataframe: ", len(data_df))
data_df.head()

Rows in Dataframe:  772


,period,stateId,stateDescription,producertypeid,producerTypeDescription,energysourceid,energySourceDescription,capability,capability-units
0,2012,TX,Texas,TOT,All sectors,ALL,All,109568.4,megawatts
1,2012,TX,Texas,TOT,All sectors,WOO,Wood,320.3,megawatts
2,2012,TX,Texas,TOT,All sectors,WND,Wind,12178.9,megawatts
3,2012,TX,Texas,TOT,All sectors,SOLPV,Solar - PV,75.2,megawatts
4,2012,TX,Texas,TOT,All sectors,SOL,Solar,75.2,megawatts


In [5]:
# Create Producer Types dataframe 
producer_types_df = data_df[["producertypeid","producerTypeDescription"]]
producer_types_df = producer_types_df.sort_values("producertypeid").drop_duplicates(ignore_index=True)
producer_types_df = producer_types_df.set_index("producertypeid")
producer_types_df

,producerTypeDescription
producertypeid,
EU,Electric Utilities
IPP,Independent Power Producers
TOT,All sectors


In [6]:
# Create Energy Sources dataframe
energy_sources_df = data_df[["energysourceid", "energySourceDescription"]]
energy_sources_df = energy_sources_df.sort_values("energysourceid").drop_duplicates(ignore_index=True)
energy_sources_df = energy_sources_df.set_index("energysourceid")
energy_sources_df

,energySourceDescription
energysourceid,
ALL,All
BAT,Battery
COL,Coal
HYC,Hydroelectric
NG,Natural Gas
NGCC,Natural Gas - CC
NGGT,Natural Gas - GT
NGIC,Natural Gas - IC
NGOTH,Natural Gas - OTH


In [7]:
# Create States dataframe
states_df = data_df[["stateId", "stateDescription"]].drop_duplicates(ignore_index=True)
states_df = states_df.set_index("stateId")
states_df

,stateDescription
stateId,
TX,Texas


In [8]:
# Drop columns to create cleaner Generating Capacity dataframe
generating_df = data_df[["period", "stateId", "producertypeid", "energysourceid", "capability", "capability-units"]]
generating_df.head()

,period,stateId,producertypeid,energysourceid,capability,capability-units
0,2012,TX,TOT,ALL,109568.4,megawatts
1,2012,TX,TOT,WOO,320.3,megawatts
2,2012,TX,TOT,WND,12178.9,megawatts
3,2012,TX,TOT,SOLPV,75.2,megawatts
4,2012,TX,TOT,SOL,75.2,megawatts


In [9]:
# Export all Dataframes to CSV for importing into SQL
energy_sources_df.to_csv("Energy_Sources_DF.csv")
producer_types_df.to_csv("Producer_Types_DF.csv")
states_df.to_csv("States_DF.csv")
generating_df.to_csv("Generating_Capacity_DF.csv")

Other Tests

In [10]:
# Generator details - January 2023 only
url = f"https://api.eia.gov/v2/electricity/operating-generator-capacity/data/?api_key={api_key}&frequency=monthly&data[0]=latitude&data[1]=longitude&facets[stateid][]=TX&start=2023-01&end=2023-01&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000"
response = requests.get(url)
response.raise_for_status()

# Retrieve the data
data2 = response.json()

    # Process the data as needed
if "data" in data2["response"]:
    # Extract the generator counts by location and energy source
    pprint.pprint(data2)

{'apiVersion': '2.1.4',
 'request': {'command': '/v2/electricity/operating-generator-capacity/data/',
             'params': {'api_key': 'x6MMEhX0n69TOHYg6HfanNHKFatn9wv68dbefLC7',
                        'data': ['latitude', 'longitude'],
                        'end': '2023-01',
                        'facets': {'stateid': ['TX']},
                        'frequency': 'monthly',
                        'length': 5000,
                        'offset': 0,
                        'sort': [{'column': 'period', 'direction': 'desc'}],
                        'start': '2023-01'}},
 'response': {'data': [{'balancing-authority-name': None,
                        'balancing_authority_code': None,
                        'energy-source-desc': 'Natural Gas',
                        'energy_source_code': 'NG',
                        'entityName': 'Austin Energy',
                        'entityid': 1015,
                        'generatorid': 'GT1',
                        'latitude': 30.3033

In [11]:
data2_df = pd.json_normalize(data2["response"]["data"])
print("Rows in Dataframe: ", len(data2_df))
data2_df.head()

Rows in Dataframe:  1821


,period,stateid,stateName,sector,sectorName,entityid,entityName,plantid,plantName,generatorid,...,energy_source_code,energy-source-desc,prime_mover_code,balancing_authority_code,balancing-authority-name,status,statusDescription,latitude,longitude,unit
0,2023-01,TX,Texas,electric-utility,Electric Utility,1015,Austin Energy,3548,Decker Creek,GT1,...,NG,Natural Gas,GT,None,None,OP,Operating,30.3033,-97.6128,None
1,2023-01,TX,Texas,electric-utility,Electric Utility,1015,Austin Energy,3548,Decker Creek,GT2,...,NG,Natural Gas,GT,None,None,OP,Operating,30.3033,-97.6128,None
2,2023-01,TX,Texas,electric-utility,Electric Utility,1015,Austin Energy,3548,Decker Creek,GT3,...,NG,Natural Gas,GT,None,None,OP,Operating,30.3033,-97.6128,None
3,2023-01,TX,Texas,industrial-chp,Industrial CHP,1182,BASF Corporation,55311,BASF Freeport Works,GEN1,...,NG,Natural Gas,GT,None,None,OP,Operating,29.0020,-95.3940,None
4,2023-01,TX,Texas,industrial-chp,Industrial CHP,1182,BASF Corporation,55311,BASF Freeport Works,GEN2,...,WH,Waste Heat,ST,None,None,OP,Operating,29.0020,-95.3940,None


In [12]:
sector_df = data2_df[["sector", "sectorName"]].drop_duplicates(ignore_index=True)
sector_df = sector_df.set_index("sector")
sector_df

,sectorName
sector,
electric-utility,Electric Utility
industrial-chp,Industrial CHP
ipp-non-chp,IPP Non-CHP
ipp-chp,IPP CHP
commercial-chp,Commercial CHP
industrial-non-chp,Industrial Non-CHP
commercial-non-chp,Commercial Non-CHP


In [13]:
entity_df = data2_df[["entityid", "entityName"]].drop_duplicates(ignore_index=True)
entity_df = entity_df.set_index("entityid")
entity_df

,entityName
entityid,
1015,Austin Energy
1182,BASF Corporation
429,Occidental Permian Ltd
1357,BASF Corp
2172,Brazos Electric Power Coop Inc
...,...
60129,"Horse Creek Wind, LLC"
60146,Ameresco Federal Solutions
60155,"Old Settler Wind, LLC"


In [14]:
plant_df = data2_df[["plantid", "plantName"]].drop_duplicates(ignore_index=True)
plant_df = plant_df.set_index("plantid")
plant_df

,plantName
plantid,
3548,Decker Creek
55311,BASF Freeport Works
7900,Sand Hill
55708,Nacogdoches Power
56374,Robert Mueller Energy Center
...,...
61887,Fort Bliss (DEA EPIC)
60366,Old Settler Wind
58488,"Bryan Solar, LLC"


In [15]:
status_df = data2_df[["status", "statusDescription"]].drop_duplicates(ignore_index=True)
status_df = status_df.set_index("status")
status_df

,statusDescription
status,
OP,Operating
SB,Standby/Backup: available for service but not ...
OS,Out of service and NOT expected to return to s...
OA,Out of service but expected to return to servi...


In [16]:
list2 = data2_df["balancing-authority-name"].unique()
list2

array([None], dtype=object)

In [17]:
generators_df = data2_df.drop(columns=["stateName", "sectorName", "entityName", "plantName", "statusDescription", "energy-source-desc", "balancing_authority_code", "balancing-authority-name", "unit"])
generators_df.head()

,period,stateid,sector,entityid,plantid,generatorid,technology,energy_source_code,prime_mover_code,status,latitude,longitude
0,2023-01,TX,electric-utility,1015,3548,GT1,Natural Gas Fired Combustion Turbine,NG,GT,OP,30.3033,-97.6128
1,2023-01,TX,electric-utility,1015,3548,GT2,Natural Gas Fired Combustion Turbine,NG,GT,OP,30.3033,-97.6128
2,2023-01,TX,electric-utility,1015,3548,GT3,Natural Gas Fired Combustion Turbine,NG,GT,OP,30.3033,-97.6128
3,2023-01,TX,industrial-chp,1182,55311,GEN1,Natural Gas Fired Combustion Turbine,NG,GT,OP,29.0020,-95.3940
4,2023-01,TX,industrial-chp,1182,55311,GEN2,All Other,WH,ST,OP,29.0020,-95.3940


In [18]:
# Export all Dataframes to CSV for importing into SQL
sector_df.to_csv("Sector_DF.csv")
entity_df.to_csv("Entity_DF.csv")
plant_df.to_csv("Plant_DF.csv")
status_df.to_csv("Status_DF.csv")
generators_df.to_csv("Generators_DF.csv")

In [21]:
list3 = generators_df["energy_source_code"].unique()
list3

array(['NG', 'WH', 'WDS', 'DFO', 'MWH', 'WND', 'WAT', 'SUN', 'OG', 'SUB',
       'LIG', 'AB', 'LFG', 'NUC', 'RC', 'PC', 'PUR', 'BLQ'], dtype=object)